TO DO List:
* context vector is stored in memory
* concatenation of context vector with the previous read list.
  * What is the size of prev_state['read_list'] = pr_size
* If size found out, x size = (batch_size, 1, embedding_size + hidden_dim+pr_Size)
* Find what the dimension of read_list should be


In [ ]:
# ## Code for generating our data ##
# import numpy as np
# import random

# def generate_seq(filename, write_in_file=True):
# 	character_sequence = []
# 	subword_sequence = []

# 	if(write_in_file):
# 		f = open("/media/aiswarya/New Volume/My_works/MANN/Model0/Data/SeqData.txt","w")
		
# 	file_ = open(filename, 'r') 
# 	lines = file_.readlines() 
# 	for line in lines:
# 		temp = line.split(':')
# 		char_seq = temp[0].strip()
# 		subwrd_seq = temp[1].strip()
# 		subword_sequence.append(subwrd_seq)
# 		character_sequence.append(char_seq)
# 		if(write_in_file):
# 		    f.write("%s\t%s\n"%(char_seq,subwrd_seq))
# 		#print("True Date %s Generated Date %s"%(char_seq,subwrd_seq))
# 	if(write_in_file):
# 		f.close()
# 	return character_sequence,subword_sequence

# def load_data(data_file):
#     f = open(data_file)
#     line = f.readline()
#     X = []
#     Y = []
#     while line:
#         info = line.strip("\n").split("\t")
#         x = info[0]
#         y = info[1]
#         X.append(x)
#         Y.append(y)
#         line = f.readline()
#     f.close()
#     return X,Y

# def find_distinct_tokens(data,separator,write_in_file, write = False):
# 	tok2ind = {}
# 	ind2tok = {}
# 	all_tok = []
# 	for d in data:
# 		info = d.split(separator)
# 		all_tok.extend(info)
		
# 	all_tok = list(set(all_tok))
# 	all_tok.sort()
# 	all_tok.extend(['<START>',"<END>"])
# 	print("all_tok")
# 	if write:
# 		f = open(write_in_file,"w")
# 		for ind,item in enumerate(all_tok):
# 			tok2ind[item] = ind
# 			ind2tok[ind] = item
# 			f.write("%s\t%d\n"%(item,ind))
# 		f.close()
# 	else:
# 		for ind,item in enumerate(all_tok):
# 			tok2ind[item] = ind
# 			ind2tok[ind] = item
# 	return tok2ind,ind2tok

# def load_tokens(token_file):
#     f = open(token_file)
#     line = f.readline()
#     tok2ind = {}
#     ind2tok = {}
#     while line:
#         info = line.strip("\n").split("\t")
#         item = info[0]
#         ind = int(info[1])
#         tok2ind[item] = ind
#         ind2tok[ind] = item
#         line = f.readline()
#     return tok2ind,ind2tok

# def string2index(datestring,tok2ind,separator):
#     components = datestring.split(separator)
#     return [tok2ind[i] for i in components]

# def batch_generator(x_data,y_data,batch_size,source_tok2ind,target_tok2ind):
#     total = len(x_data)
#     start = 0
#     i = 0
#     while i<1:
#         end = min(start + batch_size , total)
#         X = x_data[start:end]
#         Y = y_data[start:end]
#         batch_x = []
#         batch_y = []
#         for bx,by in zip(X,Y):
#             bx = string2index(bx,source_tok2ind,"-")
#             by = string2index(by,target_tok2ind,"/")
#             batch_x.append(bx)
#             batch_y.append(by)
#         #yield np.asarray(batch_x),np.asarray(batch_y),start,end
#         #print(np.asarray(batch_x), '  ',np.asarray(batch_y),'  ', start, '  ', end)
#         start = end
#         if(start >= total):
#             start = 0
            
#         i+=1
# 	return np.asarray(batch_x), np.asarray(batch_y)

# ## Test ##
# X,Y = generate_seq("/media/aiswarya/New Volume/My_works/MANN/Model0/CTC-MANN_compare_data/datafile.txt", False)
# print(len(X))
# print(len(Y))

# print(X[0], '      ', Y[0])
# print(X[100], '     ', Y[100])

# ST, SI = find_distinct_tokens(X, '-', "", False)
# TT,TI = find_distinct_tokens(Y, '/', "", False)

# #print("ST : ", ST, "Len : ", len(ST))
# #print("TT : ", TT, "Len : ", len(TT))
# bx, by = batch_generator(X,Y,5,ST,TT)
# bx = np.array(bx).flatten().tolist()
# by = np.array(by).flatten().tolist()
# for i in range(len(bx)):
# 	print(SI[i], '  ', TI[i])



In [1]:
import tensorflow as tf
import sys
import matplotlib.pyplot as plt
import os
REDUCE = 1

physical_devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)
if(physical_devices):
    tf.config.experimental.set_virtual_device_configuration(physical_devices[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])


In [66]:
import numpy as np
import random

month_names = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep",'Oct',"Nov","Dec"]

def generate_dates(N,write_in_file=False):
    #will generate random N dates in two different format
    days = np.random.randint(1,32,N)
    months = np.random.randint(1,13,N)
    years = np.random.randint(1970,2020,N)
    X = [] #used as network_input
    Y = [] #used as network target
    if(write_in_file):
        f = open("/media/aiswarya/New Volume/My_works/MANN/Model0/Data/RandomDates.txt","w")
    for a_date in zip(days,months,years):
        true_date = str(a_date[0]) + "/" + str(a_date[1]) + "/" + str(a_date[2]) 
        Y.append(true_date)
        unorder_date = [str(a_date[0]),month_names[a_date[1]-1],str(a_date[2])]
        random.shuffle(unorder_date)
        generated_date = unorder_date[0] + "-" + unorder_date[1] + "-" +unorder_date[2]
        X.append(generated_date)
        if(write_in_file):
            f.write("%s\t%s\n"%(generated_date,true_date))
        print("True Date %s Generated Date %s"%(true_date,generated_date))
    if(write_in_file):
        f.close()
    return X,Y

def load_data(data_file):
    f = open(data_file)
    line = f.readline()
    X = []
    Y = []
    while line:
        info = line.strip("\n").split("\t")
        x = info[0]
        y = info[1]
        X.append(x)
        Y.append(y)
        line = f.readline()
    f.close()
    return X,Y

def find_distinct_tokens(data,separator,write_in_file, write = False):
	tok2ind = {}
	ind2tok = {}
	all_tok = []
	for d in data:
		info = d.split(separator)
		all_tok.extend(info)
		
	all_tok = list(set(all_tok))
	all_tok.sort()
	all_tok.extend(['<START>',"<END>"])
	print("all_tok")
	if write:
		f = open(write_in_file,"w")
		for ind,item in enumerate(all_tok):
			tok2ind[item] = ind
			ind2tok[ind] = item
			f.write("%s\t%d\n"%(item,ind))
		f.close()
	else:
		for ind,item in enumerate(all_tok):
			tok2ind[item] = ind
			ind2tok[ind] = item
	return tok2ind,ind2tok

def load_tokens(token_file):
    f = open(token_file)
    line = f.readline()
    tok2ind = {}
    ind2tok = {}
    while line:
        info = line.strip("\n").split("\t")
        item = info[0]
        ind = int(info[1])
        tok2ind[item] = ind
        ind2tok[ind] = item
        line = f.readline()
    return tok2ind,ind2tok

def string2index(datestring,tok2ind,separator):
    components = datestring.split(separator)
    return [tok2ind[i] for i in components]

def batch_generator(x_data,y_data,batch_size,source_tok2ind,target_tok2ind):
    total = len(x_data)
    start = 0
    i = 0
    while True:
        end = min(start + batch_size , total)
        X = x_data[start:end]
        Y = y_data[start:end]
        batch_x = []
        batch_y = []
        for bx,by in zip(X,Y):
            bx = string2index(bx,source_tok2ind,"-")
            by = string2index(by,target_tok2ind,"/")
            batch_x.append(bx)
            batch_y.append(by)
        yield np.asarray(batch_x),np.asarray(batch_y),start,end
        #print(np.asarray(batch_x), '  ',np.asarray(batch_y),'  ', start, '  ', end)
        start = end
        if(start >= total):
            start = 0
            

## Test ##
"""
X,Y = generate_dates(10, False)
print(X)
print(Y)
ST, SI = find_distinct_tokens(X, '-', "", False)
TT,TI = find_distinct_tokens(Y, '/', "", False)

print("ST : ", SI, "Len : ", len(ST))
print("TT : ", TI, "Len : ", len(TT))
batches = batch_generator(X,Y,5,ST,TT)
"""


'\nX,Y = generate_dates(10, False)\nprint(X)\nprint(Y)\nST, SI = find_distinct_tokens(X, \'-\', "", False)\nTT,TI = find_distinct_tokens(Y, \'/\', "", False)\n\nprint("ST : ", SI, "Len : ", len(ST))\nprint("TT : ", TI, "Len : ", len(TT))\nbatches = batch_generator(X,Y,5,ST,TT)\n'

In [67]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input,LSTM,Reshape,Conv2D,Flatten, GRU

class Encoder(tf.keras.Model):

    def __init__(self, enc_units, vocab_size, embedding_dim=32, nb_layers=3):
        super(Encoder, self).__init__()
        self.enc_units = enc_units
        self.nb_layers = nb_layers
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.all_layers = []
        for l in range(self.nb_layers):
            layer = tf.keras.layers.GRU(self.enc_units, return_sequences=True,
                                        return_state=True, recurrent_initializer='glorot_uniform', dropout=0.2)
            self.all_layers.append(layer)

    def call(self, inputs, training=None, mask=None):
        x = self.embedding(inputs)
        for l in range(len(self.all_layers)):
            output, state = self.all_layers[l](x)
            x = output
        return output, state

    def initialize_hidden_state(self, batch_size):
        return tf.zeros((batch_size, self.enc_units))


class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        # print('called attention')
        # print('query shape', query.shape)
        hidden_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        # print('context vector inside attention', context_vector.shape)

        return context_vector, attention_weights


class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units,memory_size= 20,memory_vector_dim= 4,head_num= 2, nb_layers=3):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.nb_layers = nb_layers
        self.all_layers = []
        # for l in range(self.nb_layers):
        #     layer = tf.keras.layers.GRU(self.dec_units, return_sequences=True,
        #                                 return_state=True, recurrent_initializer='glorot_uniform', dropout=0.2)
        #     self.all_layers.append(layer)
        for i in range(self.nb_layers):
          layer = GRU(units=128,return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform', dropout=0.2)
          self.all_layers.append(layer)
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.memory_size = memory_size
        self.memory_vector_dim = memory_vector_dim
        self.head_num = head_num
        self.gamma = 0.95
        self.reuse = True

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)

        #weight vector for read operation   
    def read_head_addressing(self, k, prev_M):
        
        "content based cosine similarity"
        # k = tf.expand_dims(k, axis=2)
        # print('expand k :', k.shape)
        # inner_product = tf.matmul(prev_M, k)
        # k_norm = tf.sqrt(tf.reduce_sum(tf.square(k), axis=2, keepdims=True))
        # M_norm = tf.sqrt(tf.reduce_sum(tf.square(prev_M), axis=3, keepdims=True))
        # norm_product = M_norm * k_norm
        # K = tf.squeeze(inner_product / (norm_product + 1e-8))                  
        # K_exp = tf.exp(K)
        # w = K_exp / tf.reduce_sum(K_exp, axis=1, keepdims=True)      

        k = tf.expand_dims(k, axis=3)
        # print('expand k shape : ', k.shape)
        # print('M shape : ', prev_M.shape)
        inner_product = tf.matmul(prev_M, k)
        # print('inner pd shape : ', inner_product.shape)
        k_norm = tf.sqrt(tf.reduce_sum(tf.square(k), axis=2, keepdims=True))
        # print('k norm shape : ', k_norm.shape)
        M_norm = tf.sqrt(tf.reduce_sum(tf.square(prev_M), axis=3, keepdims=True))
        # print('M norm shape : ', M_norm.shape)
        norm_product = M_norm * k_norm
        # print('norm pd shape : ', norm_product.shape)
        K = tf.expand_dims(tf.squeeze(inner_product / (norm_product + 1e-8)), axis = 1)
        # print('K squeeze shape : ', K.shape)                
        K_exp = tf.exp(K)
        w = K_exp / tf.reduce_sum(K_exp, axis=1, keepdims=True)    
        # print('w shape : ', w.shape)           
                 
        
        return w

    #weight vector for write operation
    def write_head_addressing(self,sig_alpha, prev_w_r_list, prev_w_lu):
        prev_w_r = prev_w_r_list[-1]
        # print('w_r shape : ', prev_w_r.shape)
        # print('sig alp : ', sig_alpha.shape)
        # print('prev_w_lu : ', prev_w_lu.shape)
        return sig_alpha * prev_w_r + (1. - sig_alpha) * prev_w_lu     

    #least used weight vector
    def least_used(self,w_u):
        print('####')
        # print('w_u shape : ', w_u.shape)
        _, indices = tf.nn.top_k(w_u, k=self.memory_size)
        w_lu = tf.reduce_sum(tf.one_hot(indices[:, -self.head_num:], depth=self.memory_size), axis=2)
        # print('w_lu shape : ', w_lu.shape)
        return indices, w_lu


    #next we define the function called zero state for initializing all the states - 
    #controller state, read vector, weights and memory
    def zero_state(self,batch_size,dtype):
        one_hot_weight_vector = np.zeros([batch_size,1, self.memory_size])
        one_hot_weight_vector[..., 0] = 1
        one_hot_weight_vector = tf.constant(one_hot_weight_vector, dtype=tf.float32)
        with tf.compat.v1.variable_scope('init', reuse=self.reuse):
            state = {
                'read_vector_list': [tf.zeros([batch_size,1, self.memory_vector_dim])
                                      for _ in range(self.head_num)],
                'w_r_list': [one_hot_weight_vector for _ in range(self.head_num)],
                'w_u': one_hot_weight_vector,
                'M': tf.constant(np.ones([batch_size,1, self.memory_size, self.memory_vector_dim]) * 1e-6, dtype=tf.float32)
            }
            return state

    def call(self, x, hidden, enc_output, prev_state):
        # enc_output shape == (batch_size, max_length, hidden_size)
        print('initial x shape : ', x.shape)
        context_vector, attention_weights = self.attention(hidden, enc_output) #(batch_size, hidden_size)
        prev_read_vector_list = prev_state['read_vector_list']
        context_vector = tf.reshape(context_vector, (context_vector.shape[0], 1, context_vector.shape[1]))
        context_vector = tf.concat([context_vector] + prev_read_vector_list, axis=-1)
        # print('context_vector after concat', context_vector.shape)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size +prev_read_size)
        #tf.expand_dims(context_vector, 1)
        x = tf.concat([context_vector, x], axis=-1)
        print('x final shape : ', x.shape)

        # passing the concatenated vector to the GRU
        layer_output = x
        for l in range(len(self.all_layers)):

            output, state = self.all_layers[l](layer_output)
            layer_output = output

        network = Sequential()
        network.add(LSTM(units=128,activation='tanh',return_sequences=True))
        layer_output = network(layer_output)

        #layer_output = tf.reshape(layer_output, (layer_output.shape[0], layer_output.shape[2], 1))
        print('layer_output shape : ', layer_output.shape)
        num_parameters_per_head = self.memory_vector_dim + 1
        total_parameter_num = num_parameters_per_head * self.head_num
        
        #Initiliaze weight matrix and bias and compute the parameters
        weights = tf.Variable(tf.random.normal([layer_output.get_shape()[0],layer_output.get_shape()[2], total_parameter_num], stddev=0.35))
        # print('weights para : ', weights.shape)
        biases = tf.Variable(tf.zeros([total_parameter_num]))
        parameters = tf.compat.v1.nn.xw_plus_b(layer_output, weights, biases)
        # print('parameters : ', parameters.shape)
        head_parameter_list = tf.split(parameters, self.head_num, axis=2)
        # print('head parameter list : ', np.array(head_parameter_list).shape)
        
        
        #previous read weight vector
        prev_w_r_list = prev_state['w_r_list']   
        # print('w_r weights : ', prev_w_r_list[0].shape)
        
        #previous memory
        prev_M = prev_state['M']
        # print('memory : ', prev_M.shape)
        
        #previous usage weight vector
        prev_w_u = prev_state['w_u']
        # print('usage weights : ', prev_w_u.shape)
        
        #previous index and least used weight vector
        prev_indices, prev_w_lu = self.least_used(prev_w_u)
        
        #read weight vector
        w_r_list = []
        
        #write weight vector
        w_w_list = []
        
        #key vector
        k_list = []
    
        #now, we will initialize some of the important parameters that we use for addressing.     
        for i, head_parameter in enumerate(head_parameter_list):
            with tf.compat.v1.variable_scope('addressing_head_%d' % i):
                
                #key vector
                k = tf.tanh(head_parameter[:,:, 0:self.memory_vector_dim], name='k')
                # k = tf.reshape(k, (64,4))
                # print('key k : ', k)
                
                #sig_alpha
                sig_alpha = tf.sigmoid(head_parameter[:,:, -1:], name='sig_alpha')
                # sig_alpha = tf.reshape(sig_alpha, (64,1))
                # print('sig alpha : ', sig_alpha)
                
                #read weights
                w_r = self.read_head_addressing(k, prev_M)
                
                #write weights
                w_w = self.write_head_addressing(sig_alpha, prev_w_r_list[i], prev_w_lu)
           
            w_r_list.append(w_r)
            w_w_list.append(w_w)
            k_list.append(k)
            

        #usage weight vector 
        # print('w_w_list : ', w_w_list[0].shape)
        # print('w_r_list : ', w_r_list[0].shape)
        w_u = self.gamma * prev_w_u + tf.add_n(w_r_list) + tf.add_n(w_w_list)   

        #update the memory
        M_ = prev_M * tf.compat.v1.expand_dims(1. - tf.one_hot(prev_indices[:,:, -1], self.memory_size), dim=3)
        # print('M_ shape : ', M_.shape)
        
        #write operation
        M = M_
        with tf.compat.v1.variable_scope('writing'):
            for i in range(self.head_num):
                
                w = tf.expand_dims(w_w_list[i], axis=3)
                k = tf.expand_dims(k_list[i], axis=2)
                M = M + tf.matmul(w, k)

        # print('Memory shape : ', np.array(M).shape)
        #read opearion
        read_vector_list = []
        with tf.compat.v1.variable_scope('reading'):
            for i in range(self.head_num):
                read_vector = tf.reduce_sum(tf.compat.v1.expand_dims(w_r_list[i], dim=3) * M, axis=2)
                read_vector_list.append(read_vector)       

        
        #controller output
        ## printing old state shapes:
        new_state = {
            'read_vector_list': read_vector_list,
            'w_r_list': w_r_list,
            'w_w_list': w_w_list,
            'w_u': w_u,
            'M': M,
        }

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights, new_state

In [70]:
class Model_0:
    def __init__(self, name, in_vocab_size, out_vocab_size, embedding_dim, units, batch_size):
        self.name = name
        self.units = units
        self.batch_size = batch_size
        self.encoder = Encoder(units, in_vocab_size)
        print("Encoder Dimensions ", self.encoder)
        self.decoder = Decoder(out_vocab_size, embedding_dim, units)
        self.decoder_state = self.decoder.zero_state(self.batch_size, tf.float32)
        print("Decoder Dimensions ", self.decoder)
        self.optimizer = tf.keras.optimizers.Adam(
            0.0001)  # tf.keras.optimizers.RMSprop(0.0001)
        self.saver = tf.train.Checkpoint(
            encoder=self.encoder, decoder=self.decoder, optimizer=self.optimizer)
        self.save_dir = "drive/My Drive/AttentionModel/Weights/"+self.name+".ckpt"
        self.weight_manager = tf.train.CheckpointManager(
            self.saver, self.save_dir, max_to_keep=2)

    def loss_function(self, real, pred):
        loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction='none')
        loss_ = loss_object(real, pred)
        return tf.reduce_mean(loss_)

    # @tf.function
    def train_step(self, inp, targ, start_index):
        print('Inside train step')
        loss = 0
        batch_size = inp.shape[0]
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = self.encoder(inp)
            dec_hidden = enc_hidden
            dec_input = tf.expand_dims([start_index] * batch_size,1)
            #dec_input = np.array([start_index] * batch_size)
            # Teacher forcing - feeding the target as the next input
            for t in range(0, targ.shape[1]):
                # passing enc_output to the decoder
                print(t)
                predictions, dec_hidden, att_wt, state = self.decoder(dec_input, dec_hidden, enc_output, self.decoder_state)
                self.decoder_state = state
                loss += self.loss_function(targ[:, t], predictions)
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        batch_loss = (loss / int(targ.shape[1]))
        variables = self.encoder.trainable_variables + self.decoder.trainable_variables
        gradients = tape.gradient(loss, variables)
        self.optimizer.apply_gradients(zip(gradients, variables))
        return batch_loss

    def train(self, X, Y, source_tok2ind, target_tok2ind, epochs):
        restore_from = tf.train.latest_checkpoint(self.save_dir)
        self.saver.restore(restore_from)
        print('Weights restored from %s' % restore_from)

        total = len(X)
        batches = batch_generator(X, Y, self.batch_size,
                                    source_tok2ind, target_tok2ind)
        nbbatches = int(np.ceil(total / float(self.batch_size)))
        print("Ready to generate batches: Total %d #Batches %d" %
                (total, nbbatches))
        best_loss = np.inf
        print('epochs', epochs)
        for e in range(epochs):
            total_loss = 0
            print('number of batchs : ', nbbatches)
            for b in range(nbbatches):
                print('inside')
                batch_x, batch_y, start, end = next(batches)
                print('batch size : ', batch_x.shape)
                if batch_x.shape == self.batch_size:
                  b_loss = self.train_step(
                      batch_x, batch_y, start_index=target_tok2ind['<START>'])
                  total_loss += b_loss
                # sys.stdout.write("\rBatch %d/%d from %d to %d loss %0.4f" %
                #                     (b+1, nbbatches, start, end, b_loss))
                # sys.stdout.flush()
            total_loss = total_loss/nbbatches
            print("\tEpoch %d/%d Loss %0.4f" % (e+1, epochs, total_loss))
            if(total_loss < best_loss):
                self.weight_manager.save()
                best_loss = total_loss
                print("--------------New Best State-----------------")

    def predict(self, test_x, max_length_output, start_idx, source_tok2ind, target_ind2tok):
        restore_from = tf.train.latest_checkpoint(self.save_dir)
        self.saver.restore(restore_from)
        print('Weights restored from %s' % restore_from)
        bx = string2index(test_x[0], source_tok2ind, "-")
        bx = np.expand_dims(bx, 0)  # include a batch dimension
        max_length_inp = bx.shape[1]
        attention_plot = np.zeros((max_length_output, max_length_inp))

        result = ''
        enc_out, enc_hidden = self.encoder(bx)

        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([start_idx], 0)

        for t in range(max_length_output):
            predictions, dec_hidden, attention_weights = self.decoder(
                dec_input, dec_hidden, enc_out)

            # storing the attention weights to plot later on
            attention_weights = tf.reshape(attention_weights, (-1,))
            attention_plot[t] = attention_weights.numpy()

            predicted_id = tf.argmax(predictions[0]).numpy()
            predicted_char = target_ind2tok[predicted_id]
            if predicted_char == '<END>':
                return result, attention_plot

            result += predicted_char + "/"
            # the predicted ID is fed back into the model
            dec_input = tf.expand_dims([predicted_id], 0)

        return result.rstrip("/"), attention_plot


In [71]:
X, Y = generate_dates(2500)
source_tok2ind, source_ind2tok = find_distinct_tokens(X, "-", write_in_file="/media/aiswarya/New Volume/My_works/MANN/Model0/Data/source_tokens.txt", write=False)
target_tok2ind, target_ind2tok = find_distinct_tokens(Y, "/", write_in_file="/media/aiswarya/New Volume/My_works/MANN/Model0/Data/target_tokens.txt", write=False)


Nc_out = len(target_tok2ind)
print('Nc Out : ', Nc_out)
Nc_in = len(source_tok2ind)
print('Nc In : ', Nc_in)

network = Model_0('Model_0', Nc_in, Nc_out, embedding_dim=32, units=128, batch_size=64)
network.train(X, Y, source_tok2ind, target_tok2ind, 1)
# train network
"""
try:
    if sys.argv[1] == 'train':
        print('training')
        network.train(X, Y, source_tok2ind, target_tok2ind, 1)

  # now predict something
    else:
        testX, testY = generate_dates(1, write_in_file=False)
        f1 = open("Prediction_Result.txt", "a")
        print(testX)
        predicted_date, attention_mat = network.predict(
            testX, 3, target_tok2ind['<START>'], source_tok2ind, target_ind2tok)
        print(predicted_date)
        f1.write("Generated date %s \tPredited date %s\r" %
                 (str(testX), str(predicted_date)))
        f1.close()
        plt.imshow(attention_mat, cmap='Greys')
        plt.xticks(np.arange(3), predicted_date.split("/"))
        plt.yticks(np.arange(3), testX[0].split("-"))
        plt.show()
        plt.savefig('last_fig.png')
except:
    print("For trainning run:!python3 Model_0.py train")
    print("For trainning run:!python3 Model_0.py test")
"""

True Date 27/10/1978 Generated Date Oct-1978-27
True Date 5/4/1980 Generated Date 5-Apr-1980
True Date 10/1/1986 Generated Date 10-1986-Jan
True Date 13/4/1988 Generated Date Apr-1988-13
True Date 9/4/2017 Generated Date Apr-9-2017
True Date 27/11/2009 Generated Date Nov-27-2009
True Date 27/3/2011 Generated Date 2011-Mar-27
True Date 22/6/2016 Generated Date Jun-22-2016
True Date 16/1/1997 Generated Date Jan-16-1997
True Date 20/8/1974 Generated Date Aug-20-1974
True Date 15/3/2011 Generated Date 2011-Mar-15
True Date 10/10/1973 Generated Date 1973-10-Oct
True Date 26/11/2003 Generated Date Nov-2003-26
True Date 26/5/2011 Generated Date 26-May-2011
True Date 10/12/2004 Generated Date Dec-10-2004
True Date 20/1/2016 Generated Date Jan-20-2016
True Date 11/1/1976 Generated Date Jan-1976-11
True Date 18/7/2014 Generated Date Jul-2014-18
True Date 20/4/1978 Generated Date Apr-20-1978
True Date 9/7/1996 Generated Date 9-Jul-1996
True Date 23/10/1994 Generated Date 23-1994-Oct
True Date 19/

'\ntry:\n    if sys.argv[1] == \'train\':\n        print(\'training\')\n        network.train(X, Y, source_tok2ind, target_tok2ind, 1)\n\n  # now predict something\n    else:\n        testX, testY = generate_dates(1, write_in_file=False)\n        f1 = open("Prediction_Result.txt", "a")\n        print(testX)\n        predicted_date, attention_mat = network.predict(\n            testX, 3, target_tok2ind[\'<START>\'], source_tok2ind, target_ind2tok)\n        print(predicted_date)\n        f1.write("Generated date %s \tPredited date %s\r" %\n                 (str(testX), str(predicted_date)))\n        f1.close()\n        plt.imshow(attention_mat, cmap=\'Greys\')\n        plt.xticks(np.arange(3), predicted_date.split("/"))\n        plt.yticks(np.arange(3), testX[0].split("-"))\n        plt.show()\n        plt.savefig(\'last_fig.png\')\nexcept:\n    print("For trainning run:!python3 Model_0.py train")\n    print("For trainning run:!python3 Model_0.py test")\n'